In [1]:
from src.models.linreg import LinReg
from src.models.panel import FixedEffects

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

from linearmodels import PanelOLS

f = 0.1173

In [2]:
def create_panel_data():
    np.random.seed(69)
    ids = range(1, 100) 
    years = range(2010, 2021)  
    industries = ['Tech', 'Health', 'Finance', 'Education', 'Retail', 'Energy', 'Manufacturing', 'Transport', 'Services', 'Agriculture']  
    regions = ['North', 'South', 'East', 'West', 'Central']  

    industry_effects = {'Tech': 3, 'Health': 2, 'Finance': 4, 'Education': 1, 'Retail': 2, 'Energy': 5, 'Manufacturing': 3, 'Transport': 4, 'Services': 2, 'Agriculture': 3}
    region_effects = {'North': 1, 'South': 2, 'East': 1, 'West': 3, 'Central': 2}

    data_list = []
    for id in ids:
        industry = industries[id % len(industries)]  
        region = regions[id % len(regions)]  
        for year in years:
            x = np.random.uniform(0, 10)
            z = np.random.uniform(5, 15)
            industry_effect = industry_effects[industry]
            region_effect = region_effects[region]
            y = 1000 + 10* x + 5*z +10 * year + 30 * id + 40* region_effect + np.random.normal(0, 2)  
            data_list.append({'id': id, 'year': year, 'industry': industry, 'region': region, 'outcome': y, 'independent': x, 'independent2': z})

    panel_data = pd.DataFrame(data_list)
    return panel_data

panel_data = create_panel_data()
panel_data

,id,year,industry,region,outcome,independent,independent2
0,1,2010,Health,South,21307.402892,2.962492,13.090677
1,1,2011,Health,South,21312.611545,5.613490,7.535890
2,1,2012,Health,South,21270.832476,1.049771,5.584607
3,1,2013,Health,South,21379.784622,7.327832,12.878741
4,1,2014,Health,South,21296.841519,0.763763,8.176806
...,...,...,...,...,...,...,...
1084,99,2016,Agriculture,Central,24277.151056,4.086341,5.774107
1085,99,2017,Agriculture,Central,24344.327657,6.180384,11.904614
1086,99,2018,Agriculture,Central,24368.832440,7.168238,13.833236
1087,99,2019,Agriculture,Central,24287.456063,2.143771,5.177190


In [3]:

base = LinReg(df = panel_data,
       outcome='outcome',
       independent=['independent',"independent2" ]).summary(content_type='html')

In [4]:
fe1 = FixedEffects(df = panel_data,
                   outcome='outcome',
                   independent=['independent'],
                   fixed=['year'],
                   standard_error_type='clustered')

fe1.summary(content_type='html')


In [5]:
fe1.independent_vars

['independent',
 'dummy_year_2011',
 'dummy_year_2012',
 'dummy_year_2013',
 'dummy_year_2014',
 'dummy_year_2015',
 'dummy_year_2016',
 'dummy_year_2017',
 'dummy_year_2018',
 'dummy_year_2019',
 'dummy_year_2020']

In [10]:
panel_data = panel_data.set_index(['id', 'year'])

# Define the model with year-level fixed effects
# The 'TimeEffects' term accounts for year-specific fixed effects
model = PanelOLS.from_formula('outcome ~ independent + TimeEffects', data=panel_data)

# Fit the model
results = model.fit(cov_type='clustered', cluster_time=True)

# Print the results
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:                outcome   R-squared:                        0.0004
Estimator:                   PanelOLS   R-squared (Between):              0.0025
No. Observations:                1089   R-squared (Within):               0.3623
Date:                Thu, Jan 04 2024   R-squared (Overall):              0.0025
Time:                        17:14:29   Log-likelihood                   -8901.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.3867
Entities:                          99   P-value                           0.5341
Avg Obs:                       11.000   Distribution:                  F(1,1077)
Min Obs:                       11.000                                           
Max Obs:                       11.000   F-statistic (robust):             0.3390
                            

In [5]:
fe2 = FixedEffects(df = panel_data,
                   outcome='outcome',
                   independent=['independent'],
                   fixed=['year', 'id'],
                   standard_error_type='two-way-clustered')

fe2.summary(content_type='html')

getting se
getting se


In [6]:
panel_data = panel_data.set_index(['id', 'year'])

# Define the model with both time and id level fixed effects
# The 'EntityEffects' term accounts for id-specific fixed effects
# The 'TimeEffects' term accounts for year-specific fixed effects
model = PanelOLS.from_formula('outcome ~ independent + EntityEffects + TimeEffects', data=panel_data)

# Fit the model
results = model.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

# Print the results
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:                outcome   R-squared:                        0.9953
Estimator:                   PanelOLS   R-squared (Between):              0.0044
No. Observations:                1089   R-squared (Within):               0.4640
Date:                Thu, Jan 04 2024   R-squared (Overall):              0.0044
Time:                        17:22:39   Log-likelihood                   -2234.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   2.069e+05
Entities:                          99   P-value                           0.0000
Avg Obs:                       11.000   Distribution:                   F(1,979)
Min Obs:                       11.000                                           
Max Obs:                       11.000   F-statistic (robust):          3.113e+05
                            

In [ ]:
"""Year, id and region level fixed effects"""

fe3 = FixedEffects(df = panel_data,
                   outcome='outcome',
                   independent=['independent'],
                   fixed=['year', 'id', 'region']).summary(content_type='html')
